# MIBI-TOF Example with SquidPy and Napari

In [ ]:
import ark
import spatialdata as sd
from napari_spatialdata import Interactive
import spatialdata_plot
import squidpy as sq
import scanpy as sc
from pathlib import Path

In [ ]:
!mkdir ../../../data/mibitof_example_data/
!mkdir ../../../data/mibitof_example_data/mibitof_data
!wget -P ../../../data/mibitof_example_data/mibitof_data https://s3.embl.de/spatialdata/spatialdata-sandbox/mibitof.zip
!tar -xzf ../../../data/mibitof_example_data/mibitof_data/mibitof.zip -C ../../../data/mibitof_example_data/mibitof_data/.

In [ ]:
sdata = sd.read_zarr("../../../data/mibitof_example_data/mibitof_data/data.zarr")
sdata

In [ ]:
sdata.table.uns

In [ ]:
sdata.table.uns["spatialdata_attrs"]["instance_key"] = "instance_id"
sdata.table.uns["spatialdata_attrs"]["region_key"] = "region"
sdata.table.obs.rename(columns={"library_id": "region", "cell_id": "instance_id"}, inplace=True)
sdata.table.obs

## Co-occurrence across spatial dimensions

We can visualize cluster co-occurrence in spatial dimensions using the original spatial coordinates. The co-occurrence score is defined as:

$$\frac{p(A|B)}{p(A)} $$

where $p(A|B)$ is the conditional probability of observing a cluster $exp$ conditioned on the presence of a cluster $cond$, whereas $p(A)$ is the probability of observing in the radius size of interest. The score is computed across increasing radii size around each cell in the tissue.

"co-occurrence" aka conditional probability can be extended to multiple clusters as well.



In [ ]:
for fov_id, fov_sd in sdata.iter_coords:
    sq.gr.co_occurrence(fov_sd, cluster_key="Cluster")
    
    sq.pl.co_occurrence(
        fov_sd.table,
        cluster_key="Cluster",
        clusters=["Epithelial", "Imm_other", "Myeloid_CD68"],
        figsize=(15, 4),
    )

In [ ]:
sds = []

for fov_id, fov_sd in sdata.iter_coords:
    sq.gr.spatial_neighbors(fov_sd, library_key="region",)
    sc.pp.pca(fov_sd.table)
    sc.pp.neighbors(fov_sd.table)
    sc.tl.leiden(fov_sd.table)
    sq.gr.nhood_enrichment(fov_sd, cluster_key="leiden")
    sq.gr.nhood_enrichment(fov_sd, cluster_key="Cluster")
    sq.gr.centrality_scores(fov_sd, cluster_key="Cluster", score = None)
    sds.append(fov_sd)

In [ ]:
sdata2 = sd.concatenate(sds)

In [ ]:
sdata2

In [ ]:
sds[0].table

In [ ]:
Interactive(sds)

In [ ]:
sq.pl.nhood_enrichment(sds[0].table, cluster_key="leiden", figsize=(5, 5))

In [ ]:
sq.pl.nhood_enrichment(sds[0].table, cluster_key="Cluster", figsize=(5, 5))